<a href="https://colab.research.google.com/github/computational-neurology/workshop2025/blob/main/01_fMRI_and_neuropsychiatry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to fMRI, neuroanatomy and neuropsychiatry


### Let's compute functional connectivity!

Nilearn is a Python toolbox that has a built in function for extracting timeseries from functional files and doing all the extra signal processing at the same time. Let's walk through how this is done.

First we'll grab our imports to have all function that we need.

In [ ]:
#! pip install numpy
#! pip install matplotlib
#! pip install pandas
!pip install nilearn
#! pip install bids

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nilearn import image
from nilearn import datasets
from nilearn import plotting

Let's grab the data that we want to perform our connectivity analysis later on. Let’s define the regions we want to extract the signal from. We will use the Harvard-Oxford atlas.

In [ ]:
dataset = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')
atlas_filename = dataset.maps
labels = dataset.labels

Let's have a look at the brain parcellation.

In [ ]:
plotting.plot_roi(atlas_filename)

Let's now look at the names of the first ten brain regions of the parcellation.

In [ ]:
# Load labels for each atlas region
labels[:10]

We can also look at the fMRI data from the dataset. Let's have a look at the different files associated with the fMRI data.

In [ ]:
# One subject of fmri data
data = datasets.fetch_development_fmri(n_subjects=1)
func_file = data.func[0]
confound_file = data.confounds[0]

We can now have a look at the mean image of the fMRI file. You can see that it is quite blurry- the resolution is much lower than of an anatomical file.

In [ ]:
plotting.view_img(image.mean_img(func_file), threshold=None)

Now we'll import a package from <code>nilearn</code>, called <code>input_data</code> which allows us to extract the time series data using the parcellation file (i.e., the average signal over time from each of the regions of the parcellations will be extracted), and at the same time applying data preprocessing to clean the time series!

We first create an object using the parcellation file <code>Harvard-Oxford atlas</code> and our cleaning settings which are the following:

Settings to use:
- Confounds: trans_x, trans_y, trans_z, rot_x, rot_y, rot_z, white_matter, csf, global_signal
- Temporal Derivatives: Yes
- high_pass = 0.009
- low_pass = 0.08
- detrend = True
- standardize = True

To extract signal on the parcellation you can use the NiftiLabelsMasker in Python. It is a processing object that is created by specifying all the important parameters (e.g, how to preprocess the data), but not the data itself. You have to give the data to the masker later on and you can use it on as many functional imaging data as you want.

In [ ]:
# Note: if you are using Nilearn with a version newer than 0.9.0 (e.g., in the future on your local laptop),
#then you should use the following: from nilearn.maskers import NiftiLabelsMasker

from nilearn.input_data import NiftiLabelsMasker

masker = NiftiLabelsMasker(labels_img=atlas_filename,
                                      standardize=True,
                                      memory='nilearn_cache',
                                      verbose=1,
                                      detrend=True,
                                     low_pass = 0.08,
                                     high_pass = 0.009,
                                      t_r=2
                                    )


The object <code>masker</code> is now able to be used on *any functional image of the same size*. The `input_data.NiftiLabelsMasker` object is a wrapper that applies parcellation, cleaning and averaging to an functional image. For example let's apply this to our first subject:

### Using the masker
Finally with everything set up, we can now use the masker to perform our:
1. Confounds cleaning
2. Parcellation
3. Extract the average time series from one parcel (i.e., one brain region)<p>
All in one step!


In [ ]:
#Apply cleaning, parcellation and extraction to functional data
time_series = masker.fit_transform(func_file, confounds=confound_file)

Now we can have a look at the time series of one of the regions from the functional data.

In [ ]:
plt.plot(time_series[:,1])
# if you want to look at another region, just change the number 1 to another number in the code!

<div class="alert alert-block alert-success">
<b>Exercise 1</b><p>
Now let's look at the time series structure. What do these two values (168,48) mean?
    
</div>

In [ ]:
time_series.shape

The result of running <code>masker.fit_transform</code> is a matrix that has: <p>
INSERT ANSWER HERE
- Rows matching the XXXX (168)
- Columns matching the XXXX (48)

<div class="alert alert-block alert-success">
-End of Exercise-
            </div>

We originally had **49 ROIs**, what happened to 1 of them? It turns out that <code>masker</code> drops ROIs that are empty (i.e contain no brain voxels inside of them), this means that 1 of our atlas' parcels did not correspond to any region with signal! To see which ROIs are kept after computing a parcellation you can look at the <code>labels_</code> property of <code>masker</code>:

In [ ]:
print(masker.labels_)
print("Number of labels", len(masker.labels_))

If you want to work with multiple subjects (all of whom might have different missing values), you should create an array that contains the all regions (and has zeros for the missing values). This is not part of our class today.

### Calculating Connectivity


We now have 48 time series from 48 regions, so we can see which regions have similar activity over time, i.e., are functionally connected to each other. This does not mean that they are directly connected via fibers, but they can be also connected via another region.

<div class="alert alert-block alert-success">
<b>Exercise 2</b><p>

There are different types of connectivity that we will discuss:
    1. ROI-based connectivity
    2. Independent component analysis.
Please have a look inside this paper: http://www.ajnr.org/content/early/2018/01/18/ajnr.A5527 and explain the difference between these two.
    
    
-End of Exercise-  
</div>

In our seminar today, we will focus on the ROI-based connectivity, i.e. we will see how each region (of interest = ROI) from a parcellation is functionally connected with each other. We will calculate a *full connectivity matrix* by computing the correlation between *all pairs of ROIs* in our parcellation scheme!

We'll use another nilearn tool called <code>ConnectivityMeasure</code> from <code>nilearn.connectome</code>. This tool will perform the full set of pairwise correlations for us

Like the masker, we need to make an object that will calculate connectivity for us.

In [ ]:
from nilearn.connectome import ConnectivityMeasure

In [ ]:
correlation_measure = ConnectivityMeasure(kind='correlation')

Then we use <code>correlation_measure.fit_transform()</code> in order to calculate the full correlation matrix for our parcellated data!


<div class="alert alert-block alert-success">
<b>Exercise 3</b><p>

Please now include the missing variable (i.e., the preprocessed time series from above) in the code below to calculate the connectivity.

</div>

In [ ]:
full_correlation_matrix = correlation_measure.fit_transform([])

<div class="alert alert-block alert-success">
-End of Exercise-  
</div>

<div class="alert alert-block alert-success">
<b>Exercise 3a (optional)</b><p>

If you have time, how can you calcuate functional connectivity without a Nilearn function (i.e., just the correlations of the time series)? E.g., which Numpy or scikit-learn functions would be suitable to retrieve pairwise correlations or even ICA?

</div>

In [ ]:
import sklearn

<div class="alert alert-block alert-success">
-End of Exercise-  
</div>

<div class="alert alert-block alert-success">
<b>Exercise 4</b><p>
Let's now look at the shape of the resulting connectivity matrix. What does it mean?
    
INSERT ANSWER HERE

In [ ]:
full_correlation_matrix.shape

<div class="alert alert-block alert-success">
-End of Exercise-  
</div>

<div class="alert alert-block alert-success">
<b>Exercise 5</b><p>

Below you find a visualization of the connectivity matrix. How would you describe it?
    
INSERT ANSWER HERE
</div>

In [ ]:
plt.imshow(np.squeeze(full_correlation_matrix))

<div class="alert alert-block alert-success">
-End of Exercise-  
</div>

<div class="alert alert-block alert-success">
<b>Exercise 6</b><p>

Now that you know functional connectivity. It gives a value for each pair of brain regions over time. Can you split your team into two and one half does a search on  dynamic functional connectivity and the other on effective connectivity and then explains to the others? What are its advantages and disadvantages?
    
INSERT YOUR THOUGHTS HERE

-End of Exercise-  
</div>

## Congratulations!

Hopefully now you understand that:

1. fMRI data needs to be pre-processed before analyzing
2. Manipulating images in python is easily done using `nilearn` and `nibabel`
3. You can also do post-processing like confound/nuisance regression using `nilearn`
4. Parcellating is a method of simplifying and "averaging" data. The type of parcellation reflect assumptions you make about the structure of your data
5. Functional Connectivity is really just time-series correlations between two signals!

## More information/additional reading material:

https://andysbrainbook.readthedocs.io/en/latest/

https://nilearn.github.io/user_guide.html